# Unzipping a File from One Location to Another

In [1]:
import shutil
import os

In [2]:
src = '/kaggle/input/continue-video-frame-extraction'
dst = '/kaggle/working/'

# Copy all files and folders
for item in os.listdir(src):
    s = os.path.join(src, item)
    d = os.path.join(dst, item)
    if os.path.isdir(s):
        shutil.copytree(s, d, dirs_exist_ok=True)  # Copy entire folder
    else:
        shutil.copy2(s, d)  # Copy file

---

# Counting Subdirectories for Each Folder in a List

In [3]:
import os

In [4]:
# List of folder paths
folder_list = [
    '/kaggle/working/preprocessed_frames/train',
    '/kaggle/working/preprocessed_frames/dev',
    '/kaggle/working/preprocessed_frames/test'
]

# Count subdirectories for each folder
for folder in folder_list:
    if os.path.exists(folder):
        subdirs = [d for d in os.listdir(folder) if os.path.isdir(os.path.join(folder, d))]
        count = len(subdirs)
        print(f"{folder}: {count} subdirectories")
    else:
        print(f"{folder}: Path does not exist")

/kaggle/working/preprocessed_frames/train: 2539 subdirectories
/kaggle/working/preprocessed_frames/dev: Path does not exist
/kaggle/working/preprocessed_frames/test: Path does not exist


---

# Remove Unwanted Files by Extension from a Directory

In [5]:
folder = '/kaggle/working/'
extensions_to_remove = ['html','.css', '.bz2','.ipynb','.dat', 'txt', 'bak1', 'bak2']  # Add all the extensions you want to remove

# Loop through all files
for filename in os.listdir(folder):
    if any(filename.endswith(ext) for ext in extensions_to_remove):
        file_path = os.path.join(folder, filename)
        os.remove(file_path)
        print(f"Deleted: {file_path}")

Deleted: /kaggle/working/__notebook__.ipynb
Deleted: /kaggle/working/__results__.html
Deleted: /kaggle/working/custom.css
Deleted: /kaggle/working/mmod_human_face_detector.dat.bz2
Deleted: /kaggle/working/shape_predictor_68_face_landmarks.dat
Deleted: /kaggle/working/shape_predictor_68_face_landmarks.dat.bz2
Deleted: /kaggle/working/mmod_human_face_detector.dat


---

# Delete Specific Files by Full Path

In [6]:
# List of full file paths you want to delete
files_to_remove = [
    '/kaggle/working/__output__.json',
    '/kaggle/working/frame_count.txt',
    '/kaggle/working/__output__.json'
]

# Loop through and remove each file
for file_path in files_to_remove:
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Deleted: {file_path}")
    else:
        print(f"File not found, skipped: {file_path}")

Deleted: /kaggle/working/__output__.json
File not found, skipped: /kaggle/working/frame_count.txt
File not found, skipped: /kaggle/working/__output__.json


---

# Remove a Specific Directory Using Shell Command

In [7]:
!rm -r /kaggle/working/__results___files

rm: cannot remove '/kaggle/working/__results___files': No such file or directory


---

# Essential Data Processing Steps

In [8]:
# Install dependencies
!pip install face_recognition opencv-python-headless==4.5.5.64
try:
    import dlib
    if not dlib.DLIB_USE_CUDA:
        !pip install dlib
except ImportError:
    !pip install dlib
!wget -q http://dlib.net/files/mmod_human_face_detector.dat.bz2
!bzip2 -dk mmod_human_face_detector.dat.bz2
!wget -q http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 MB 37.1 MB/s eta 0:00:00
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=f3244100bc56364fe8830120740897aa0bbf81dd841d2f7275fa55046e0fd6a3
  Stored in directory: /root/.cache/pip/wheels/04/52/ec/9355da79c29f160b038a20c784db2803c2f9fa2c8a462c176a
Successfully built face-recognition-models
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.4 requires opencv-python-headless

In [9]:
# Add this diagnostic check after imports:
print(f"DLIB CUDA Status: {dlib.DLIB_USE_CUDA}")
print(f"Available CUDA Devices: {dlib.cuda.get_num_devices()}")

DLIB CUDA Status: True
Available CUDA Devices: 1


In [10]:
# Enhanced Imports
import cv2
import json
import signal
import numpy as np
import face_recognition
import dlib
from tqdm import tqdm
from collections import deque
from scipy.signal import find_peaks
import time
import tempfile
import tarfile
import fcntl
import sys
import threading
import os
import shutil

In [11]:
class Config:
    # Path configurations
    input_json = "/kaggle/input/meld-emotion-recognition/JSON files/JSON files/Final Format/MELD_Video_Data_Cleaned.json"
    output_root = "/kaggle/working/preprocessed_frames"
    checkpoint_file = "/kaggle/working/extraction_checkpoint.json"
    emergency_dir = "/kaggle/working/emergency_backups"
    
    # Processing parameters
    num_frames = 32
    crop_size = (224, 224)
    face_confidence = 0.6
    speaker_history = 30
    similarity_threshold = 0.4
    centrality_weight = 0.2
    min_speaking_frames = 15
    bbox_expand_ratio = 0.7
    tracking_quality_threshold = 7.0
    max_tracking_failures = 5

    # System parameters
    max_retries = 3
    checkpoint_backups = 2
    frame_validation_threshold = 32
    safety_margin = 30 * 60
    max_runtime = 12 * 60 * 60 - safety_margin
    
    # GPU optimization (NEW)
    use_gpu = dlib.DLIB_USE_CUDA
    gpu_batch_size = 128 if use_gpu else 1

In [12]:
class AtomicFrameSaver:
    @staticmethod
    def save_frame(frame, path):
        """Atomic file write with retries and correct extensions"""
        base, ext = os.path.splitext(path)
        temp_path = f"{base}.tmp{ext}"
        for attempt in range(Config.max_retries):
            try:
                os.makedirs(os.path.dirname(path), exist_ok=True)
                success = cv2.imwrite(temp_path, frame)
                if not success:
                    raise RuntimeError(f"OpenCV failed to write {temp_path}")
                os.rename(temp_path, path)
                return True
            except Exception as e:
                if attempt == Config.max_retries - 1:
                    if os.path.exists(temp_path):
                        os.remove(temp_path)
                    raise RuntimeError(f"Failed to save {path} after {Config.max_retries} attempts: {str(e)}")
                time.sleep(0.1 * (attempt + 1))

In [13]:
class MouthAnalyzer:
    def __init__(self):
        self.mar_history = deque(maxlen=Config.speaker_history)
        
    def calculate_rhythm_score(self):
        if len(self.mar_history) < 10:
            return 0
        mar_values = np.array(self.mar_history)
        eps = 1e-8
        mar_norm = (mar_values - np.mean(mar_values)) / (np.std(mar_values) + eps)
        peaks, _ = find_peaks(mar_norm, prominence=0.5)
        valley, _ = find_peaks(-mar_norm, prominence=0.5)
        if len(peaks) > 2 and len(valley) > 2:
            peak_intervals = np.diff(peaks)
            rhythm_consistency = 1 - np.std(peak_intervals) / np.mean(peak_intervals)
            return min(max(rhythm_consistency, 0), 1)
        return 0

In [14]:
class AtomicCheckpointManager:
    @staticmethod
    def load():
        try:
            if os.path.exists(Config.checkpoint_file):
                with open(Config.checkpoint_file, 'r') as f:
                    return json.load(f)
            emergency_checkpoint = os.path.join(Config.emergency_dir, "checkpoint.json")
            if os.path.exists(emergency_checkpoint):
                shutil.copy(emergency_checkpoint, Config.checkpoint_file)
                return AtomicCheckpointManager.load()
        except Exception as e:
            print(f"Checkpoint load failed: {str(e)}")
        return {"processed": {"train": [], "dev": [], "test": []}, "metadata": {"train": [], "dev": [], "test": []}}

    @staticmethod
    def save(data):
        os.makedirs(Config.emergency_dir, exist_ok=True)
        temp_path = f"{Config.checkpoint_file}.tmp"
        with open(temp_path, 'w') as f:
            json.dump(data, f)
            f.flush()
            os.fsync(f.fileno())
        os.replace(temp_path, Config.checkpoint_file)
        shutil.copy(Config.checkpoint_file, os.path.join(Config.emergency_dir, "checkpoint.json"))

In [15]:
class SpeakerTracker:
    def __init__(self):
        self.known_faces = {}
        self.current_focus = None
        self.focus_confidence = 0
        self.consecutive_frames = 0
        self.face_analyzers = {}
        
    def update_focus(self, fid, score, frame_size):
        x_center = self.known_faces[fid]['position'][0] + self.known_faces[fid]['position'][2]/2
        y_center = self.known_faces[fid]['position'][1] + self.known_faces[fid]['position'][3]/2
        centrality = 1 - (abs(x_center - frame_size[1]/2)/(frame_size[1]/2) +
                          abs(y_center - frame_size[0]/2)/(frame_size[0]/2))/2
        combined_score = (score * (1 - Config.centrality_weight) + 
                        centrality * Config.centrality_weight)
        
        if self.current_focus == fid:
            confidence_gain = combined_score * 0.1
            self.focus_confidence = min(self.focus_confidence + confidence_gain, 1)
            self.consecutive_frames += 1
        else:
            confidence_gain = combined_score * 0.05
            self.focus_confidence = max(self.focus_confidence - 0.02, 0)
            
        if (combined_score > self.focus_confidence + 0.2 or 
            (self.consecutive_frames < Config.min_speaking_frames and 
             combined_score > self.focus_confidence)):
            self.current_focus = fid
            self.focus_confidence = combined_score
            self.consecutive_frames = 1

In [16]:
def identify_main_speaker(video_path):
    cap = cv2.VideoCapture(video_path)
    tracker = SpeakerTracker()
    frame_size = (int(cap.get(4)), int(cap.get(3)))
    detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        dlib_faces = detector(rgb, 1)
        faces = []
        for face in dlib_faces:
            if face.confidence >= Config.face_confidence:
                rect = face.rect
                faces.append((rect.top(), rect.right(), rect.bottom(), rect.left()))
        
        encodings = face_recognition.face_encodings(rgb, faces)
        current_faces = []
        
        for (top, right, bottom, left), encoding in zip(faces, encodings):
            face_id = None
            for fid, data in tracker.known_faces.items():
                similarity = 1 - face_recognition.face_distance([data['encoding']], encoding)[0]
                if similarity > Config.similarity_threshold:
                    face_id = fid
                    break
                    
            if face_id is None:
                face_id = len(tracker.known_faces)
                tracker.known_faces[face_id] = {
                    'encoding': encoding,
                    'analyzer': MouthAnalyzer(),
                    'position': (left, top, right-left, bottom-top)
                }
                
            shape = predictor(rgb, dlib.rectangle(left, top, right, bottom))
            mar = get_mouth_aspect_ratio(shape)
            tracker.known_faces[face_id]['analyzer'].mar_history.append(mar)
            current_faces.append(face_id)
            
        scores = {}
        for fid in current_faces:
            data = tracker.known_faces[fid]
            rhythm_score = data['analyzer'].calculate_rhythm_score()
            recent_mar = np.mean(list(data['analyzer'].mar_history)[-10:])
            activity_score = recent_mar * (1 + rhythm_score)
            scores[fid] = activity_score
            
        if scores:
            max_fid = max(scores, key=lambda x: scores[x])
            tracker.update_focus(max_fid, scores[max_fid], frame_size)
            
    cap.release()
    
    if tracker.consecutive_frames >= Config.min_speaking_frames and tracker.current_focus is not None:
        return tracker.known_faces[tracker.current_focus]['encoding']
    return None

In [17]:
def get_mouth_aspect_ratio(shape):
    try:
        points = [(shape.part(i).x, shape.part(i).y) for i in range(48, 68)]
        vert_dist = (np.linalg.norm(np.array(points[2]) - np.array(points[10])) + 
                   np.linalg.norm(np.array(points[4]) - np.array(points[8]))) / 2
        horiz_dist = np.linalg.norm(np.array(points[0]) - np.array(points[6]))
        return vert_dist / horiz_dist if horiz_dist != 0 else 0
    except:
        return 0

In [18]:
# Global emergency stop flag
EMERGENCY_STOP = threading.Event()

In [19]:
class GracefulTerminator:
    def __init__(self):
        self.start_time = time.time()
        self.time_limit = self.start_time + Config.max_runtime
        print(f"⏰ Auto-save at: {time.ctime(self.time_limit)}")

    def should_stop(self):
        return EMERGENCY_STOP.is_set() or (time.time() > self.time_limit)

    def remaining(self):
        return max(0, self.time_limit - time.time())

In [20]:
def get_mask_info(output_dir):
    """Generate mask_info list based on existing frame files"""
    mask_info = []
    for idx in range(Config.num_frames):
        frame_num = f"{idx:02d}"
        umf_path = os.path.join(output_dir, f"{frame_num}_UMF.jpg")
        mask_info.append(1 if os.path.exists(umf_path) else 0)
    return mask_info

In [21]:
def process_dataset():

    try:
        # First-run initialization
        if not os.path.exists(Config.checkpoint_file):
            print("🚀 Starting first-run processing")
            # Create fresh checkpoint structure
            checkpoint = {
                "processed": {"train": [], "dev": [], "test": []},
                "metadata": {"train": [], "dev": [], "test": []}
            }
            AtomicCheckpointManager.save(checkpoint)
        else:
            previous_run_path = "/kaggle/input/continue-video-frame-extraction"
            if os.path.exists(previous_run_path):
                print("🚚 Loading previous run data...")
                try:
                    if os.path.exists(Config.output_root):
                        print(f"🔧 Cleaning existing output directory {Config.output_root}")
                        shutil.rmtree(Config.output_root)
                    if os.path.exists(Config.checkpoint_file):
                        print(f"🔧 Removing existing checkpoint {Config.checkpoint_file}")
                        os.remove(Config.checkpoint_file)
    
                    src_checkpoint = os.path.join(previous_run_path, "extraction_checkpoint.json")
                    if os.path.exists(src_checkpoint):
                        print(f"⚙️ Copying checkpoint from {src_checkpoint}")
                        shutil.copy(src_checkpoint, Config.checkpoint_file)
    
                    src_frames = os.path.join(previous_run_path, "preprocessed_frames")
                    if os.path.exists(src_frames):
                        print(f"⚙️ Merging frames from {src_frames}")
                        shutil.copytree(
                            src_frames,
                            Config.output_root,
                            dirs_exist_ok=True
                        )
                    print("✅ Previous run data loaded successfully")
                except Exception as e:
                    print(f"❌ Failed to load previous run: {str(e)}")
                    raise

        # ALWAYS load input data
        with open(Config.input_json) as f:
            original_data = json.load(f)
            assert all(split in original_data for split in ['train', 'dev', 'test']), \
            "Invalid JSON structure - missing required splits"

        checkpoint = AtomicCheckpointManager.load()
        terminator = GracefulTerminator()


        
        for split in ['train', 'dev', 'test']:
            split_dir = os.path.join(Config.output_root, split)
            os.makedirs(split_dir, exist_ok=True)
                

        # IMPROVEMENT: Calculate initial counts per split
        initial_counts = {
            split: len(checkpoint["processed"][split])
            for split in ['train', 'dev', 'test']
        }


        
        # IMPROVEMENT : Cross-split progress tracking
        total_videos = sum(len(original_data[s]) for s in ['train', 'dev', 'test'])
        main_pbar = tqdm(total=total_videos, initial=sum(initial_counts.values()), desc="Total Progress")

        
        # Main processing logic
        for split in ['train', 'dev', 'test']:
            split_dir = os.path.join(Config.output_root, split)
            os.makedirs(split_dir, exist_ok=True)

            # Get processed videos for this split
            processed_videos = set(checkpoint["processed"][split])
            total_in_split = len(original_data[split])

            # Create filtered list and progress bar
            unprocessed_items = [
                item for item in original_data[split]
                if [k for k in item.keys() if k not in ('y', 'label')][0] not in processed_videos
            ]

            # CORRECTED: Track progress through full dataset
            split_pbar = tqdm(
                unprocessed_items,
                desc=f"Processing {split}",
                total=total_in_split,
                initial=initial_counts[split]
            )
            
            
            for item in original_data[split]:
                if terminator.should_stop():
                    print("🛑 Emergency stop triggered - terminating")
                    raise SystemExit
                
                # Extract video key (e.g., "0_0")
                video_key = [k for k in item.keys() if k not in ('y', 'label')][0]
                video_path = item[video_key]
                output_dir = os.path.join(split_dir, video_key)

                # Skip already processed WITHOUT updating progress bar
                if video_key in processed_videos:
                    continue
                    
                
                try:
                    
                    if os.path.exists(output_dir):
                        existing_frames = len([
                            f for f in os.listdir(output_dir) 
                            if f.endswith(".jpg") and not f.startswith("_temp")
                        ])
                        if existing_frames == Config.frame_validation_threshold:
                
                            # IMPROVEMENT : Immediate checkpoint update
                            print(f"⏩ Skipping {video_key} (has {existing_frames} valid frames)")

                            mask_info = get_mask_info(output_dir)  # Changed line
                            
                            checkpoint["metadata"][split].append({
                                **item,
                                "frames_dir": output_dir,
                                "mask_info": mask_info  # Now using actual validation
                            })
                            
                            checkpoint["processed"][split].append(video_key)
                            AtomicCheckpointManager.save(checkpoint)
                            split_pbar.update(1)
                            main_pbar.update(1)
                            continue
                        shutil.rmtree(output_dir)
                    
                    mask_info = process_video(video_path, output_dir)
                    
                    checkpoint["metadata"][split].append({
                        **item,
                        "frames_dir": output_dir,
                        "mask_info": mask_info
                    })
                    checkpoint["processed"][split].append(video_key)
                    
                    if len(checkpoint["processed"][split]) % 5 == 0:
                        AtomicCheckpointManager.save(checkpoint)
                        print(f"💾 Saved checkpoint after {len(checkpoint['processed'][split])} {split} videos")
                    
                    # IMPROVEMENT 3: Update main progress bar
                    main_pbar.update(1)
                    
                except Exception as e:
                    print(f"❌ Error processing {video_key}: {str(e)}")
                    if os.path.exists(output_dir):
                        shutil.rmtree(output_dir)
                    raise

            
            split_pbar.close()
            AtomicCheckpointManager.save(checkpoint)
            
            print(f"💾 Saved final checkpoint for {split} split")

        main_pbar.close()
        
        print("✅ All processing completed successfully!")
        return checkpoint

    except SystemExit:
        print("🌀 Cleaning up after emergency stop")
        AtomicCheckpointManager.save(checkpoint)
        raise
    except Exception as e:
        print(f"❌ Unexpected error: {str(e)}")
        raise

In [22]:
def handle_session_end(signum, frame):
    lock_file = "/kaggle/working/.save_lock"
    lock_fd = None
    
    try:
        lock_fd = os.open(lock_file, os.O_CREAT | os.O_WRONLY)
        try:
            fcntl.flock(lock_fd, fcntl.LOCK_EX | fcntl.LOCK_NB)
        except BlockingIOError:
            print("⏳ Save already in progress - skipping duplicate")
            return

        print(f"\n⚠️ Session ending! Saving state (Signal {signum})")
        
        if not os.path.exists(Config.checkpoint_file):
            print("🔄 No checkpoint to save")
            return

        with tempfile.TemporaryDirectory() as tmpdir:
            tmp_checkpoint = os.path.join(tmpdir, "checkpoint.json")
            shutil.copy(Config.checkpoint_file, tmp_checkpoint)
            
            tmp_tar = os.path.join(tmpdir, "emergency.tar.gz")
            with tarfile.open(tmp_tar, "w:gz") as tar:
                tar.add(Config.output_root, arcname="preprocessed_frames")
                tar.add(tmp_checkpoint, arcname="checkpoint.json")
            
            shutil.move(tmp_tar, "/kaggle/working/emergency_snapshot.tar.gz")
            print("✅ Emergency save completed")

    except Exception as e:
        print(f"❌ Emergency save failed: {str(e)}")
    finally:
        if lock_fd is not None:
            try:
                fcntl.flock(lock_fd, fcntl.LOCK_UN)
                os.close(lock_fd)
            except OSError:
                pass
        try:
            os.remove(lock_file)
        except FileNotFoundError:
            pass
        EMERGENCY_STOP.set()
        sys.exit(1)

In [23]:
def resume_processing():
    # First-run detection
    if not os.path.exists(Config.checkpoint_file):
        print("🆕 No existing checkpoint - starting fresh processing")
        process_dataset()
        return
    
    # Existing resume logic
    signal.signal(signal.SIGTERM, handle_session_end)
    signal.signal(signal.SIGHUP, handle_session_end)
    signal.signal(signal.SIGINT, handle_session_end)
    
    process_dataset()

---

# Improved Speaker Detection Mechanism

In [24]:
def process_video(video_path, output_dir):
    temp_dir = f"{output_dir}_processing"
    final_dir = output_dir
    cap = None
    
    try:
        os.makedirs(temp_dir, exist_ok=True)
        os.makedirs(final_dir, exist_ok=True)
        mask_info = []
        
        speaker_encoding = identify_main_speaker(video_path)
        
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise RuntimeError(f"Could not open video {video_path}")
            
        frames = []
        confirmed_focus = None
        detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')
        
        tracker = None
        tracking_failures = 0
        last_valid_face = None
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
                
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            current_face = None
            
            if tracker is not None:
                tracking_quality = tracker.update(rgb)
                if tracking_quality >= Config.tracking_quality_threshold:
                    tracked_pos = tracker.get_position()
                    t = int(tracked_pos.top())
                    r = int(tracked_pos.right())
                    b = int(tracked_pos.bottom())
                    l = int(tracked_pos.left())
                    current_face = (t, r, b, l)
                    tracking_failures = 0
                    last_valid_face = current_face
                else:
                    tracking_failures += 1
                    if tracking_failures > Config.max_tracking_failures:
                        tracker = None
                        tracking_failures = 0
            
            if current_face is None:
                dlib_faces = detector(rgb, 1)
                valid_faces = []
                for face in dlib_faces:
                    if face.confidence >= Config.face_confidence:
                        rect = face.rect
                        valid_faces.append((rect.top(), rect.right(), rect.bottom(), rect.left()))
                
                best_face = None
                best_similarity = -1
                
                if speaker_encoding is not None:
                    for (t, r, b, l) in valid_faces:
                        encoding = face_recognition.face_encodings(rgb, [(t, r, b, l)])[0]
                        similarity = 1 - face_recognition.face_distance([speaker_encoding], encoding)[0]
                        if similarity > best_similarity and similarity >= Config.similarity_threshold:
                            best_similarity = similarity
                            best_face = (t, r, b, l)
                
                if best_face:
                    t, r, b, l = best_face
                    current_face = (t, r, b, l)
                    tracker = dlib.correlation_tracker()
                    tracker.start_track(rgb, dlib.rectangle(l, t, r, b))
                    last_valid_face = current_face
                elif last_valid_face:
                    current_face = last_valid_face
                elif valid_faces:
                    frame_center = (rgb.shape[1]//2, rgb.shape[0]//2)
                    closest_dist = float('inf')
                    for face in valid_faces:
                        t, r, b, l = face
                        face_center = ((l + r)//2, (t + b)//2)
                        dist = (face_center[0]-frame_center[0])**2 + (face_center[1]-frame_center[1])**2
                        if dist < closest_dist:
                            closest_dist = dist
                            current_face = face
                    last_valid_face = current_face
                else:
                    current_face = None
            
            if current_face:
                t, r, b, l = current_face
            elif confirmed_focus:
                x, y, w, h = confirmed_focus
                l, t, r, b = x, y, x + w, y + h
            else:
                t, r, b, l = 0, frame.shape[1], frame.shape[0], 0
            
            orig_height = b - t
            orig_width = r - l
            new_t = int(max(0, t - orig_height * Config.bbox_expand_ratio))
            new_b = int(min(frame.shape[0], b + orig_height * Config.bbox_expand_ratio))
            new_l = int(max(0, l - orig_width * Config.bbox_expand_ratio))
            new_r = int(min(frame.shape[1], r + orig_width * Config.bbox_expand_ratio))
            
            cropped = frame[new_t:new_b, new_l:new_r]
            resized = cv2.resize(cropped, Config.crop_size)
            frames.append(resized)
            confirmed_focus = (new_l, new_t, new_r - new_l, new_b - new_t)
        
        cap.release()
    
        valid_frames = [f for f in frames if np.mean(f) > 10]
        for idx in range(Config.num_frames):
            frame = valid_frames[idx] if idx < len(valid_frames) else np.zeros((*Config.crop_size, 3), dtype=np.uint8)
            frame_type = "UMF" if idx < len(valid_frames) else "MF"
            output_path = f"{final_dir}/{idx:02d}_{frame_type}.jpg"
            AtomicFrameSaver.save_frame(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR), output_path)
            mask_info.append(1 if frame_type == "UMF" else 0)
        
        return mask_info
        
    except Exception as e:
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)
        raise
    finally:
        if cap is not None:
            cap.release()
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

---

# Processing the Input Data

In [25]:
if __name__ == "__main__":
    resume_processing()
    !find {Config.output_root} -name "*.jpg" | wc -l > /kaggle/working/frame_count.txt
    print("📊 Final frame count:")
    !cat /kaggle/working/frame_count.txt

🚚 Loading previous run data...
🔧 Cleaning existing output directory /kaggle/working/preprocessed_frames
🔧 Removing existing checkpoint /kaggle/working/extraction_checkpoint.json
⚙️ Copying checkpoint from /kaggle/input/continue-video-frame-extraction/extraction_checkpoint.json
⚙️ Merging frames from /kaggle/input/continue-video-frame-extraction/preprocessed_frames
✅ Previous run data loaded successfully
⏰ Auto-save at: Sun May  4 06:51:58 2025


Total Progress:  19%|█▊        | 2540/13706 [00:17<53:03:59, 17.11s/it]

💾 Saved checkpoint after 2540 train videos


Total Progress:  19%|█▊        | 2545/13706 [02:26<79:41:59, 25.71s/it]

💾 Saved checkpoint after 2545 train videos


...

💾 Saved checkpoint after 5255 train videos


Total Progress:  38%|███▊      | 5258/13706 [11:30:00<56:50:01, 24.22s/it]

🛑 Emergency stop triggered - terminating
🌀 Cleaning up after emergency stop


SystemExit: 

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
